In [6]:
from poitousprint import Portic, Toflit
import json
import networkx as nx
from ipysigma import Sigma

portic_client = Portic()
toflit_client = Toflit()

# this function allows to map a value from a domain of min-max to another
def map_value(value, domain_min, domain_max, range_min, range_max):
    left_span = domain_max - domain_min
    right_span = range_max - range_min

    # Convert the left range into a 0-1 range (float)
    scaled = float(value - domain_min) / float(left_span)

    # Convert the 0-1 range into a value in the right range.
    return range_min + (scaled * right_span)

#### Fonction générique qui permet de fabriquer un réseau à partir :

1. d'une liste de dicts (ex. flux toflit18)
2. d'une liste de deux propriétés à comparer

In [7]:
def render_coocurrences_graph(data, key_1, key_2, params=None):
    # créer un graphe
    Graph = nx.Graph()

    # créer des dict pour les deux types de noeuds et les liens
    key1_uniq = {}
    key2_uniq = {}
    edges_uniq = {}
    default_params = {
        "color_1": "rgb(0, 255, 0)", # noeuds de type 1 en vert
        "color_2": "rgb(255, 0, 0)", # noeuds de type 2 en rouge
        "node_min_size": 1,
        "node_max_size": 10
    }
    final_params = default_params
    if params is not None :
        final_params = {
            *default_params,
            *params
        }
    
    # remplir les dicts
    for datum in data:
        if key_1 in datum and key_2 in datum:
            value_1 = datum[key_1] if datum[key_1] is not None else "undefined"
            value_2 = datum[key_2] if datum[key_2] is not None else "undefined"
            value_1_id = key_1 + "_" + value_1
            value_2_id = key_2 + "_" + value_2
            if value_1_id in key1_uniq:
                key1_uniq[value_1_id] = {**key1_uniq[value_1_id], "size": key1_uniq[value_1_id]["size"] + 1}
            else:
               key1_uniq[value_1_id] = {
                   "type": key_1, 
                   "name": value_1, 
                   "color": final_params["color_1"],
                   "size": 1
               }
            if value_2_id in key2_uniq:
                key2_uniq[value_2_id] = {**key2_uniq[value_2_id], "size": key2_uniq[value_2_id]["size"] + 1}
            else:
               key2_uniq[value_2_id] = {
                   "type": key_2, 
                   "name": value_2, 
                   "color": final_params["color_2"],
                   "size": 1
               }
            edge_footprint = value_1_id + "-" + value_2_id
            if edge_footprint in edges_uniq:
                edges_uniq[edge_footprint]["weight"] += 1
            else:
                edges_uniq[edge_footprint] = {
                    "source": value_1_id,
                    "target": value_2_id,
                    "weight": 1
                }
    # concaténer les deux dicts de noeuds en un seul
    all_nodes = key1_uniq
    all_nodes.update(key2_uniq)
    # applatir et formatter les noeuds
    nodes = []
    for key, node in all_nodes.items():
        nodes.append((key, node))
    edges = []

    for key, edge in edges_uniq.items():
        edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))
        
    # ajuster la taille des noeuds en fonction d'un min et d'un max donnés
    domain_min_nodes_size = min([node[1]['size'] for node in nodes])
    domain_max_nodes_size = max([node[1]['size'] for node in nodes])
    range_in_nodes_size = [final_params["node_min_size"], final_params["node_max_size"]]
    nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

    for node in nodes:
        node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
        node[1]["label"] = node[1]["name"]


    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(edges)

    return Sigma(Graph, start_layout=True)

# 0. Je vais chercher les pointcalls / flows / travels du sprint (côté Navigo) pour nourrir mes réseaux

In [8]:
pointcalls = portic_client.get_pointcalls(source_subset = 'Poitou_1789')

In [14]:
pointcalls[0]

{'pkid': 93362,
 'record_id': '00349916',
 'pointcall': 'Arzon',
 'pointcall_uhgs_id': 'A0145524',
 'toponyme_fr': 'Arzon',
 'toponyme_en': 'Arzon',
 'latitude': '47.549033',
 'longitude': '-2.893367',
 'pointcall_admiralty': 'Vannes',
 'pointcall_province': 'Bretagne',
 'pointcall_states': '[{"1749-1815" : "France"}]',
 'pointcall_substates': None,
 'pointcall_states_en': '[{"1749-1815" : "France"}]',
 'pointcall_substates_en': None,
 'state_1789_fr': 'France',
 'state_1789_en': 'France',
 'substate_1789_fr': None,
 'substate_1789_en': None,
 'source_1787_available': False,
 'source_1789_available': False,
 'pointcall_status': None,
 'shiparea': 'ACE-IROI',
 'pointcall_point': '0101000020110F0000D5408190A0A813C166F399CFC6025741',
 'ferme_direction': None,
 'ferme_bureau': None,
 'ferme_bureau_uncertainty': None,
 'partner_balance_1789': None,
 'partner_balance_supp_1789': 'France',
 'partner_balance_1789_uncertainty': None,
 'partner_balance_supp_1789_uncertainty': 0.0,
 'pointcall_ou

In [9]:
len(pointcalls)

13829

In [17]:
flows = portic_client.get_flows(
        ports = ['A1964694', # je fonctionne avec tous les UHGS_id des ports associés à la direction des fermes de La Rochelle => fournit par Christine (à vérifier)
                'A0171758',
                'A0136930',
                'A0196496',
                'A0198999',
                'A0137148',
                'A0127055',
                'A0133403',
                'A0213721',
                'A0199508',
                'A0148208',
                'A0141325',
                'A0138533',
                'A1964982',
                'A0186515',
                'A0124809',
                'A1964767',
                'A0172590',
                'A0181608',
                'A0165077',
                'A0169240',
                'A0165056',
                'A1963997',
                'A0136403',
                'A0195938',
                'A0122971',
                'A0207992'], # sinon on devrait pouvoir fonctionner avec source_subset = 'Poitou_1789' mais pour l'instant ne fonctionne pas pour moi
    year = 1789)

In [18]:
flows[0]

{'travel_id': '0000138N- 07',
 'distance_dep_dest': 221.509,
 'distance_homeport_dep': None,
 'departure': 'La Rochelle',
 'departure_fr': 'La Rochelle',
 'departure_en': 'La Rochelle',
 'departure_uhgs_id': 'A0198999',
 'departure_latitude': '46.166667',
 'departure_longitude': '-1.15',
 'departure_admiralty': 'La Rochelle',
 'departure_province': 'Aunis',
 'departure_states': '[{"1749-1815" : "France"}]',
 'departure_substates': None,
 'departure_state_1789_fr': 'France',
 'departure_substate_1789_fr': None,
 'departure_state_1789_en': 'France',
 'departure_substate_1789_en': None,
 'departure_ferme_direction': 'La Rochelle',
 'departure_ferme_bureau': 'La Rochelle',
 'departure_ferme_bureau_uncertainty': 0.0,
 'departure_partner_balance_1789': None,
 'departure_partner_balance_supp_1789': 'France',
 'departure_partner_balance_1789_uncertainty': None,
 'departure_partner_balance_supp_1789_uncertainty': 0,
 'departure_shiparea': 'ACE-ROCH',
 'departure_status': 'siège amirauté',
 'dep

In [19]:
len(flows)

7045

In [20]:
travels = portic_client.get_travels(source_subset = 'Poitou_1789')

In [7]:
travels[0]

{'travel_id': '0004361N- 16',
 'distance_dep_dest': 102.282,
 'distance_homeport_dep': 379.809,
 'departure': 'Ribérou [Saujon]',
 'departure_fr': 'Ribérou',
 'departure_en': 'Ribérou',
 'departure_uhgs_id': 'A1964767',
 'departure_latitude': '45.6755',
 'departure_longitude': '-0.935826',
 'departure_admiralty': 'Marennes',
 'departure_province': 'Saintonge',
 'departure_states': '[{"1749-1815" : "France"}]',
 'departure_substates': None,
 'departure_state_1789_fr': 'France',
 'departure_substate_1789_fr': None,
 'departure_state_1789_en': 'France',
 'departure_substate_1789_en': None,
 'departure_ferme_direction': 'La Rochelle',
 'departure_ferme_bureau': 'Marennes',
 'departure_ferme_bureau_uncertainty': 0.0,
 'departure_partner_balance_1789': None,
 'departure_partner_balance_supp_1789': 'France',
 'departure_partner_balance_1789_uncertainty': None,
 'departure_partner_balance_supp_1789_uncertainty': 0.0,
 'departure_shiparea': 'ACE-ROCH',
 'departure_status': 'oblique',
 'departur

In [22]:
len(travels)

890

# 1. Réseaux bipartites : ports de départ / ports d'arrivée

Si 2 ports font partie du même flow, ou du même travel, ils seront reliés sur le graphe

Remarques : 
- Peut-être intéressant de filtrer les ports de départ / d'arrivée sur la direction des Fermes de La Rochelle (pour savoir où vont les bateaux qui partent de la région / d'où viennent les bateaux arrivent dans la région)
- Avec les flows risque de duplications
- Avec travels on a pas beaucoup de données apparemment



In [21]:
render_coocurrences_graph(flows, "departure_fr", "destination_fr")

# ports de départ en vert
# ports d'arrivée en rouge

Sigma(data={'nodes': [('departure_fr_La Rochelle', {'type': 'departure_fr', 'name': 'La Rochelle', 'color': 'r…

In [23]:
render_coocurrences_graph(travels, "departure_fr", "destination_fr")

# ports de départ en vert
# ports d'arrivée en rouge

Sigma(data={'nodes': [('departure_fr_Ribérou', {'type': 'departure_fr', 'name': 'Ribérou', 'color': 'rgb(0, 25…

# 2. Réseau monopartite : échanges entre ports

### EN CHANTIER : je tente d'éviter la duplication de noeuds pour un même port mais n'y arrive pas pour l'instant


In [35]:
def render_monopartite_graph_port_exchanges(data): # data could be Navigo flows or travels 

    # créer un graphe
    Graph = nx.Graph()

    # créer des dict pour l'unique type de noeuds et les liens
    harbours_uniq = {}
    edges_uniq = {}

    # remplir les dicts
    for datum in data:
        harbour1 = datum["departure_fr"]
        harbour2 = datum["destination_fr"]
        harbour_id1 = "harbour_" + harbour1
        harbour_id2 = "harbour_" + harbour2

        if harbour_id1 in harbours_uniq:
            harbours_uniq[harbour_id1] = {**harbours_uniq[harbour_id1], "size": harbours_uniq[harbour_id1]["size"] + 1}
        else:
           harbours_uniq[harbour_id1] = {
               "type": "harbour", 
               "name": harbour1, 
               "color": "rgb(0, 255, 0)",
               "size": 1
           }

        if harbour_id2 in harbours_uniq:
            harbours_uniq[harbour_id2] = {**harbours_uniq[harbour_id1], "size": harbours_uniq[harbour_id2]["size"] + 1}
        else:
           harbours_uniq[harbour_id2] = {
               "type": "harbour", 
               "name": harbour2, 
               "color": "rgb(0, 255, 0)",
               "size": 1
           }

        edge_footprint = harbour_id1 + "-" + harbour_id2

        if edge_footprint in edges_uniq:
            edges_uniq[edge_footprint]["weight"] += 1
        else:
            edges_uniq[edge_footprint] = {
                "source": harbour_id1,
                "target": harbour_id2,
                "weight": 1
            }

    # concaténer les deux dicts de noeuds en un seul => obsolète dans ce cas
    all_nodes = harbours_uniq
    # all_nodes.update(# autre dict de noeuds )

    # applatir et formatter les noeuds
    nodes = []
    for key, node in all_nodes.items():
        nodes.append((key, node))
    edges = []

    for key, edge in edges_uniq.items():
        edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))

    domain_min_nodes_size = min([node[1]['size'] for node in nodes])
    domain_max_nodes_size = max([node[1]['size'] for node in nodes])
    range_in_nodes_size = [1, 10]
    nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

    for node in nodes:
        node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
        node[1]["label"] = node[1]["name"]


    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(edges)

    return Sigma(Graph)

In [36]:
render_monopartite_graph_port_exchanges(flows)

Sigma(data={'nodes': [('harbour_La Rochelle', {'type': 'harbour', 'name': 'Ribérou', 'color': 'rgb(0, 255, 0)'…

In [37]:
render_monopartite_graph_port_exchanges(travels)

Sigma(data={'nodes': [('harbour_Ribérou', {'type': 'harbour', 'name': 'Esnandes', 'color': 'rgb(0, 255, 0)', '…